# Sample ETL skill

In [1]:
import pandas as pd
import sqlite3

## Part 1: Reformat data for db inclusion

Before creating your SQLite database, I need to clean/reformat data from several federal sources. All the data that you need is in `./..data/raw/`<sup id="a1">[1](#f1)</sup>. Clean and reformat the data so that it can be included in the database. Only include data from 2017 onward.

A simple SQLite database with the following tables:

| table | file(s) | contains |
|-------|----------------------|-|
| district_race | schools_ccd_lea_enrollment_20* | The total number students of each race for each `year` and `leaid` |
| district_grade| schools_ccd_lea_enrollment_20* | The total number students of each grade for each `year` and `leaid` |
| district_sex | schools_ccd_lea_enrollment_20* | The total number students of each sex for each `year` and `leaid` |
| district_finance | districts_ccd_finance.csv | All finance information you choose to include for each `year` and `leaid` |
| district_testScores | districts_edfacts_assessments_20* | All test information you choose to include for each `year` and `leaid` |
| district_population | districts_saipe.csv | All population information you choose to include for each `year` and `leaid` |

<b id="f1">1</b>: Download the data from our Google Drive [here](https://drive.google.com/drive/folders/1WToXvY0_oIKe7Ptq7VY6nz4fRi17U1Gu?usp=sharing) and place them in the `./../data/raw` directory manually. A dictionary to help you understand these files can be found [here](https://educationdata.urban.org/documentation/school-districts.html).

In [2]:
### YOUR CODE HERE
import os
path = '../data/raw'
files = os.listdir(path)
files

['districts_ccd_finance.csv',
 'districts_edfacts_assessments_2017.csv',
 'districts_edfacts_assessments_2018.csv',
 'districts_edfacts_assessments_2020.csv',
 'districts_saipe.csv',
 'schools_ccd_lea_enrollment_2017.csv',
 'schools_ccd_lea_enrollment_2018.csv',
 'schools_ccd_lea_enrollment_2019.csv',
 'schools_ccd_lea_enrollment_2020.csv',
 'schools_ccd_lea_enrollment_2021.csv',
 'school_districts_lea_directory.csv']

# Data Transformation

### District Finance

In [3]:
# Feature of interests
col_interests = ['year', 'leaid', 'rev_local_private_contrib','rev_total','exp_total'
                 ,'outlay_capital_total', 'enrollment_fall_responsible','enrollment_fall_school']

# Load data
full_df = pd.read_csv(path + '/districts_ccd_finance.csv', dtype={'leaid':'str'})
df = pd.read_csv(path + '/districts_ccd_finance.csv', usecols = col_interests, dtype={'leaid':'str'})

# Calculate percentage of missing values in each column
missing_values = df.isnull().mean() * 100

# Create a new DataFrame with column names and their respective missing value percentages
missing_df = pd.DataFrame({'column_name': missing_values.index, 'missing_percentage': missing_values.values})

# Missing Values Pct
missing_df

,column_name,missing_percentage
0,year,0.000000
1,leaid,0.000000
2,rev_total,0.000000
3,rev_local_private_contrib,45.213334
4,exp_total,0.000000
5,outlay_capital_total,0.000000
6,enrollment_fall_responsible,0.013667
7,enrollment_fall_school,59.542745


In [4]:
df['year'].unique()

array([1989, 1991, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018], dtype=int64)

In [5]:
# Handling missing values in 'rev_local_private_contrib'
null_vals = df['rev_local_private_contrib'].isnull().sum()
print(f'Number of null values: {null_vals}')
zero_vals = df[df['rev_local_private_contrib'] == 0]['leaid'].count()
other_special_vals = df[df['rev_local_private_contrib'] < 0]['leaid'].count()
print(f'Number of zeros: {zero_vals}')
print(f'Number of special values: {other_special_vals}')

Number of null values: 208419
Number of zeros: 127934
Number of special values: 17157


In [6]:
# Fill null with 0 and replace anything less than 0 with 0
df['rev_local_private_contrib'] = df['rev_local_private_contrib'].fillna(0)
df['rev_local_private_contrib'] = df['rev_local_private_contrib'].clip(lower = 0)
df['rev_total'] = df['rev_total'].clip(lower = 0)

In [7]:
# Last check for null values
df.isnull().sum().sum()

274536

In [8]:
# Check for duplicate
print(f'There are {df.duplicated().sum()} duplicated rows in the sub df')
print(f'There are {full_df.duplicated().sum()} duplicated rows in the full df')

There are 139 duplicated rows in the sub df
There are 0 duplicated rows in the full df


There are 216 duplicated rows in the small df and there are no duplications in the full dataset so these duplication is not true duplication. 

The data dictionary has indicated there are records with missing/not reported (-1) and not applicable data (-2). I am going to combine their revenue and rev_local_private_contrib into id '0000000'

In [9]:
# Add column that count number of districts
df['num_district'] = 1

# Change non-specified id to generic value
df['leaid'] = df['leaid'].replace(['-1','-2','-3'],'0000000')

#Combine total_rev and rev_local_private_contrib
df = df.groupby(['year','leaid']).agg({'rev_total':'sum',
                                       'rev_local_private_contrib':'sum', 
                                       'num_district' :'sum',
                                       'exp_total':'sum',
                                       'outlay_capital_total':'sum', 
                                       'enrollment_fall_responsible':'sum',
                                       'enrollment_fall_school':'sum'})
df = df.reset_index()
df

,year,leaid,rev_total,rev_local_private_contrib,num_district,exp_total,outlay_capital_total,enrollment_fall_responsible,enrollment_fall_school
0,1989,0000000,663678000.0,0.0,114,657718000.0,41057000.0,37150.0,0.0
1,1989,0100005,12933000.0,0.0,1,11703000.0,1570000.0,3120.0,0.0
2,1989,0100006,19699000.0,0.0,1,22140000.0,3734000.0,5511.0,0.0
3,1989,0100007,25090000.0,0.0,1,31101000.0,5618000.0,5879.0,0.0
4,1989,0100030,12431000.0,0.0,1,12079000.0,979000.0,3797.0,0.0
...,...,...,...,...,...,...,...,...,...
459301,2018,5680252,0.0,0.0,1,-2.0,-2.0,34.0,34.0
459302,2018,5680253,0.0,0.0,1,-2.0,-2.0,0.0,0.0
459303,2018,5680254,0.0,0.0,1,-2.0,-2.0,240.0,240.0
459304,2018,5680255,0.0,0.0,1,-2.0,-2.0,29.0,29.0


In [10]:
# Export the DataFrame to a CSV file
df.to_csv('../data/derived/districts_ccd_finance_transformed.csv', index=False)

## District Race, Grade, Gender 

In [11]:
import re

# Get all files
pattern = re.compile(r'^schools_ccd_lea_enrollment_20')
file_names = [file for file in files if re.match(pattern, file)]

# Read all files into a DataFrame
df_rgg = []
for file in file_names:
    f = pd.read_csv(path+'/'+file, dtype={'leaid':'str'})
    df_rgg.append(f)
df_rgg = pd.concat(df_rgg, ignore_index = True)
df_rgg.head()

,year,leaid,fips,grade,race,sex,enrollment
0,2017,0100002,1,10,5,2,NaN
1,2017,0100002,1,10,5,1,NaN
2,2017,0100002,1,10,4,2,NaN
3,2017,0100002,1,10,4,1,NaN
4,2017,0100002,1,10,2,2,NaN


In [12]:
df_rgg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31856877 entries, 0 to 31856876
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   year        int64  
 1   leaid       object 
 2   fips        int64  
 3   grade       int64  
 4   race        int64  
 5   sex         int64  
 6   enrollment  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 1.7+ GB


In [13]:
# Check for null values
print(f'Total null values: {df_rgg.isna().sum().sum()}')

print('Percentage of null values in each feature')
df_rgg.isna().mean() * 100

Total null values: 4619228
Percentage of null values in each feature


year           0.00000
leaid          0.00000
fips           0.00000
grade          0.00000
race           0.00000
sex            0.00000
enrollment    14.49994
dtype: float64

In [14]:
# mean enrollment by year, leaid, grade, race, sex
mean_enrollment = df_rgg.groupby(['year','leaid', 'grade','race'])['enrollment'].mean().reset_index()

# Merge the mean enrollment values back into the original DataFrame
df_rgg = pd.merge(df_rgg, mean_enrollment, on=['year','leaid','grade','race'], how='left', suffixes=('', '_mean'))

# Fill missing Enrollment values with the mean values
df_rgg['enrollment'] = df_rgg['enrollment'].fillna(df_rgg['enrollment_mean'].astype(int))

# Drop the temporary mean enrollment column
df_rgg = df_rgg.drop(columns=['enrollment_mean'])

df_rgg

,year,leaid,fips,grade,race,sex,enrollment
0,2017,0100002,1,10,5,2,0.0
1,2017,0100002,1,10,5,1,0.0
2,2017,0100002,1,10,4,2,0.0
3,2017,0100002,1,10,4,1,0.0
4,2017,0100002,1,10,2,2,0.0
...,...,...,...,...,...,...,...
31856872,2021,7800030,78,99,4,99,59.0
31856873,2021,7800030,78,99,5,99,8.0
31856874,2021,7800030,78,99,6,99,1.0
31856875,2021,7800030,78,99,7,99,18.0


In [15]:
df_rgg.isna().sum().sum()

0

In [16]:
## Count rows where enrollment is -1,-2,-3
special_val_count = df_rgg[df_rgg['enrollment'] < 0]['leaid'].count()
print(f'Number of district where enrollment value is not valid: {special_val_count}')

Number of district where enrollment value is not valid: 0


In [17]:
# Check for duplication
df_rgg.duplicated().sum()

0

In [18]:
# Export transformed data to csv
district_race_df = df_rgg.groupby(['year','leaid','race'])['enrollment'].sum().reset_index()
district_race_df.to_csv('../data/derived/districts_race_transformed.csv', index=False)

district_grade_df = df_rgg.groupby(['year','leaid','grade'])['enrollment'].sum().reset_index()
district_grade_df.to_csv('../data/derived/districts_grade_transformed.csv', index=False)

district_sex_df = df_rgg.groupby(['year','leaid','sex'])['enrollment'].sum().reset_index()
district_sex_df.to_csv('../data/derived/districts_sex_transformed.csv', index=False)

## Test Score

In [19]:
# Get all files
pattern = re.compile(r'^districts_edfacts_assessments_20')
file_names = [file for file in files if re.match(pattern, file)]

# Read all files into a DataFrame
df_ts = []
for file in file_names:
    f = pd.read_csv(path+'/'+file, dtype={'leaid':'str'})
    df_ts.append(f)
df_ts = pd.concat(df_ts, ignore_index = True)
df_ts.head()

,leaid_num,leaid,year,lea_name,fips,grade_edfacts,race,sex,lep,homeless,...,foster_care,military_connected,read_test_num_valid,read_test_pct_prof_low,read_test_pct_prof_high,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_low,math_test_pct_prof_high,math_test_pct_prof_midpt
0,100005,0100005,2017,Albertville City,1,99,99,99,99,99,...,99,99,2885.0,38.0,38.0,38.0,2892.0,45.0,45.0,45.0
1,100005,0100005,2017,Albertville City,1,99,5,99,99,99,...,99,99,5.0,-3.0,-3.0,-3.0,5.0,-3.0,-3.0,-3.0
2,100005,0100005,2017,Albertville City,1,99,4,99,99,99,...,99,99,18.0,40.0,59.0,49.5,18.0,60.0,79.0,69.5
3,100005,0100005,2017,Albertville City,1,99,2,99,99,99,...,99,99,98.0,30.0,34.0,32.0,98.0,30.0,34.0,32.0
4,100005,0100005,2017,Albertville City,1,99,99,99,99,99,...,99,99,197.0,15.0,19.0,17.0,196.0,15.0,19.0,17.0


In [20]:
df_ts = df_ts.drop(['leaid_num', 'lea_name', 'fips',
              'read_test_pct_prof_low','read_test_pct_prof_high','read_test_pct_prof_midpt',
             'math_test_pct_prof_low', 'math_test_pct_prof_high', 'math_test_pct_prof_midpt'], axis = 1)
df_ts = df_ts.dropna(axis = 0)

In [21]:
df_ts.isna().sum()

leaid                  0
year                   0
grade_edfacts          0
race                   0
sex                    0
lep                    0
homeless               0
migrant                0
disability             0
econ_disadvantaged     0
foster_care            0
military_connected     0
read_test_num_valid    0
math_test_num_valid    0
dtype: int64

In [22]:
df_ts.duplicated().sum()

0

In [23]:
df_ts.to_csv('../data/derived/districts_test_score_transformed.csv', index = False)

## District Info

In [24]:
df_dist = pd.read_csv(path+'/districts_saipe.csv', dtype = {'leaid':'str'})
df_dist.head()

,district_id,district_name,est_population_total,est_population_5_17,est_population_5_17_poverty,year,leaid,fips,est_population_5_17_poverty_pct,est_population_5_17_pct
0,5,ALBERTVILLE CITY SCH DIST,16294,2779,506,1995,0100005,1,0.182080,0.170554
1,30,ALEXANDER CITY CITY SCH DIST,17704,3258,850,1995,0100030,1,0.260896,0.184026
2,60,ANDALUSIA CITY SCH DIST,9602,1706,571,1995,0100060,1,0.334701,0.177671
3,90,ANNISTON CITY SCH DIST,27394,4907,2015,1995,0100090,1,0.410638,0.179127
4,100,ARAB CITY SCH DIST,7984,1299,223,1995,0100100,1,0.171671,0.162700


In [25]:
df_dist['est_population_5_17_poverty_pct'] = df_dist['est_population_5_17_poverty_pct'].fillna(0)
df_dist['est_population_5_17_pct'] = df_dist['est_population_5_17_pct'].fillna(0)
df_dist.isna().sum()

district_id                        0
district_name                      0
est_population_total               0
est_population_5_17                0
est_population_5_17_poverty        0
year                               0
leaid                              0
fips                               0
est_population_5_17_poverty_pct    0
est_population_5_17_pct            0
dtype: int64

In [26]:
df_dist.duplicated().sum()

0

In [27]:
df_dist.to_csv('../data/derived/districts_saipe_transformed.csv', index = False)

## Part 2: Create the database

Create your database here. Save it to `./../data/derived/district_data.db`

In [28]:
### YOUR CODE HERE
import sqlite3
import csv

# Path to the CSV file
finance_file = '../data/derived/districts_ccd_finance_transformed.csv'
race_file = '../data/derived/districts_race_transformed.csv'
grade_file = '../data/derived/districts_grade_transformed.csv'
sex_file = '../data/derived/districts_sex_transformed.csv'
testscore_file = '../data/derived/districts_test_score_transformed.csv'
dist_file = '../data/derived/districts_saipe_transformed.csv'

# Path to the SQLite database file
db_file = '../data/derived/district_data.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# FINANCE
# Create a table to store the data
cursor.execute('''CREATE TABLE IF NOT EXISTS district_finance (
                    year INTEGER,
                    leaid VARCHAR(7),
                    rev_total FLOAT,
                    rev_local_private_contrib FLOAT,
                    num_district INTEGER,
                    exp_total   FLOAT,
                    outlay_capital_total FLOAT, 
                    enrollment_fall_responsible INTEGER,
                    enrollment_fall_school INTEGER                    
                  )''')

# Read the CSV file and insert data into the database
with open(finance_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO district_finance (year, leaid, rev_total, rev_local_private_contrib, num_district, exp_total, outlay_capital_total, enrollment_fall_responsible,enrollment_fall_school) VALUES (?, ?, ?, ?, ?,?,?,?,?)', row)

#RACE
cursor.execute('''CREATE TABLE IF NOT EXISTS district_race (
                    year INTEGER,
                    leaid VARCHAR(7),
                    race INTEGER,
                    enrollment INTEGER
                  )''')

# Read the CSV file and insert data into the database
with open(race_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO district_race (year, leaid, race, enrollment) VALUES (?, ?, ?, ?)', row)

#GRADE
cursor.execute('''CREATE TABLE IF NOT EXISTS district_grade (
                    year INTEGER,
                    leaid VARCHAR(7),
                    grade INTEGER,
                    enrollment INTEGER
                  )''')

# Read the CSV file and insert data into the database
with open(grade_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO district_grade (year, leaid, grade, enrollment) VALUES (?, ?, ?, ?)', row)

#GENDER
cursor.execute('''CREATE TABLE IF NOT EXISTS district_sex (
                    year INTEGER,
                    leaid VARCHAR(7),
                    sex INTEGER,
                    enrollment INTEGER
                  )''')

# Read the CSV file and insert data into the database
with open(sex_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO district_sex (year, leaid, sex, enrollment) VALUES (?, ?, ?, ?)', row)

#TEST SCORE
cursor.execute('''CREATE TABLE IF NOT EXISTS district_testscores (
                    leaid  VARCHAR(7),
                    year INTEGER,
                    grade_edfacts INTEGER,
                    race INTEGER,
                    sex INTEGER,
                    lep INTEGER,
                    homeless INTEGER,
                    migrant INTEGER,
                    disability INTEGER,
                    econ_disadvantaged INTEGER,
                    foster_care INTEGER,
                    military_connected INTEGER,
                    read_test_num_valid INTEGER,
                    math_test_num_valid INTEGER
                  )''')

# Read the CSV file and insert data into the database
with open(testscore_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('''INSERT INTO district_testscores (leaid, year, grade_edfacts, race ,
                    sex, lep, homeless, migrant ,
                    disability ,
                    econ_disadvantaged ,
                    foster_care ,
                    military_connected ,
                    read_test_num_valid ,
                    math_test_num_valid ) VALUES (?, ?, ?, ?,?,?,?,?,?,?,?,?,?,?)''', row)

#POPULATION
cursor.execute('''CREATE TABLE IF NOT EXISTS district_pop (district_id INTEGER, 
district_name VARCHAR(128), est_population_total INTERGER, est_population_5_17  INTEGER,
est_population_5_17_poverty INTEGER, year INTEGER, leaid VARCHAR(7), fips INTEGER, 
est_population_5_17_poverty_pct FLOAT, est_population_5_17_pct FLOAT)''')

# Read the CSV file and insert data into the database
with open(dist_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('''INSERT INTO district_pop (district_id, district_name,
                                                            est_population_total,
                                                            est_population_5_17,
                                                            est_population_5_17_poverty,
                                                            year, leaid, fips,
                                                            est_population_5_17_poverty_pct,
                                                            est_population_5_17_pct ) VALUES (?, ?, ?, ?,?,?,?,?,?,?)''', row)
        
# Commit the changes and close the connection
conn.commit()
conn.close()

# Confirm data has been loaded by reading from the database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('SELECT * FROM district_finance LIMIT 5')
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

(1989, '0000000', 663678000.0, 0.0, 114, 657718000.0, 41057000.0, 37150, 0)
(1989, '0100005', 12933000.0, 0.0, 1, 11703000.0, 1570000.0, 3120, 0)
(1989, '0100006', 19699000.0, 0.0, 1, 22140000.0, 3734000.0, 5511, 0)
(1989, '0100007', 25090000.0, 0.0, 1, 31101000.0, 5618000.0, 5879, 0)
(1989, '0100030', 12431000.0, 0.0, 1, 12079000.0, 979000.0, 3797, 0)


## Part 3: Query database

```
# How many students were enrolled in leaid 0100006 in 2017?

database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    f.enrollment_fall_school AS enrollment
FROM 
    district_pop p
INNER JOIN 
    district_finance f ON p.leaid = f.leaid
WHERE 
    p.leaid = '0100006'
    AND f.year = 2017
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)
```
| leaid | district_name | fips | enrollment |
| - | - | - | - |
| 100006 | Marshall County School District | 1 | 5662.0 |

In [ ]:
# How many students were enrolled in leaid 0100006 in 2017?
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    f.enrollment_fall_school AS enrollment
FROM 
    district_pop p
INNER JOIN 
    district_finance f ON p.leaid = f.leaid
WHERE 
    p.leaid = '0100006'
    AND f.year = 2017
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)



     leaid             district_name  fips  enrollment
0  0100006  MARSHALL COUNTY SCH DIST     1        5662


In [30]:
# what is the district with the highest ratio of white students to black students in third grade in year 2021?
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    f.enrollment_fall_school AS enrollment
FROM 
    district_pop p
INNER JOIN 
    district_finance f ON p.leaid = f.leaid
WHERE 
    p.leaid = '0100006'
    AND f.year = 2017
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)
### YOUR CODE HERE

     leaid             district_name  fips  enrollment
0  0100006  MARSHALL COUNTY SCH DIST     1        5662


In [31]:
# What district brought in the most revenue in 2021? ("rev_total")
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    sum(f.rev_total) rev_total
FROM 
    district_pop p
INNER JOIN 
    district_finance f ON p.leaid = f.leaid
WHERE 
    f.year = 2021
GROUP BY 
    p.leaid, 
    p.district_name,
    p.fips
ORDER BY rev_total DESC
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)


Empty DataFrame
Columns: [leaid, district_name, fips, rev_total]
Index: []


Finance Data goes up until 2018

In [32]:
# What district brought in the most revenue in 2018? ("rev_total")
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    sum(f.rev_total) rev_total
FROM 
    district_pop p
INNER JOIN 
    district_finance f ON p.leaid = f.leaid
WHERE 
    f.year = 2018
GROUP BY 
    p.leaid, 
    p.district_name,
    p.fips
ORDER BY rev_total DESC
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)


     leaid                          district_name  fips     rev_total
0  3620580  New York City Department Of Education    36  1.757003e+12


In [33]:
# what district has the greatest delta between the number of students deemed proficient in reading versus math? (read_test_num_valid, math_test_num_valid)
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    p.leaid, 
    p.district_name,
    p.fips,
    t.read_test_num_valid - t.math_test_num_valid delta
FROM 
    district_pop p
INNER JOIN 
    district_testscores t ON p.leaid = t.leaid
ORDER BY delta DESC
LIMIT 1;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)
### YOUR CODE HERE


     leaid                  district_name  fips  delta
0  1201440  ORANGE COUNTY SCHOOL DISTRICT    12  14812


In [34]:
# What is the mean amount of private contributions for a district to collect? ("rev_local_private_contrib")
# just print the mean. There won't be an leaid, district_name, or fips id.
database_path = './../data/derived/district_data.db'
conn = sqlite3.connect(database_path)
query = """
SELECT 
    ROUND(AVG(rev_local_private_contrib),2) average_rev_local_private_contributions
FROM 
    district_finance
WHERE 
    rev_local_private_contrib > 0;
"""

df_result = pd.read_sql_query(query, conn)
conn.close()
print(df_result)
### YOUR CODE HERE

   average_rev_local_private_contributions
0                                145451.51
